In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 필요한 라이브러리 설치
!pip install langchain-community
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install tiktoken
!pip install langchain-google-genai
!pip install chromadb
!pip install langchain_huggingface
!pip install ragas

In [3]:
import os
import pandas as pd
import bs4
import tiktoken
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

In [4]:
# gemini
YOUR_API_KEY = ''
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

In [5]:
# test set - 일단 50개만
file_path = '/content/drive/MyDrive/MINI AIFFELTHON/data/test.csv'

# UTF-8 인코딩
test = pd.read_csv(file_path, encoding='utf-8')

In [6]:
test.head()

,query,answer
0,LG 전자의 AI 기반 스마트 홈 기술이 CES 2025에서 어떻게 소개되었는지 알...,[CES 2025] 노트북 켰더니 AI가 알아서 '척척'… LG 제안 '공감지능'의...
1,우리나라 국방부나 군사 강국에서 AI와 IoT를 적용한 방산기술이나 트렌드에 관한 ...,"""중요한 건 AI야""... 미군과 글로벌 방산업계가 꼽았다\nhttps://n.ne..."
2,우리나라 기업들이 AI 기술을 활용해서 일상의 큰 변화를 일으킨 것에 대한 기사가 ...,어쩌다 만나던 AI가 이젠 한집안 식구가 돼 가는 현실 [CES 2025]\nhtt...
3,"페르소나 AI는 뭐야? 또, 온디바이스 AI 기술은 뭐야? IoT는 네트워크랑 연결...","페르소나AI, 인터넷·GPU 필요없는 ‘온디바이스 AI’로 CES 2025 혁신상 ..."
4,LG 전자에서 발표한 AI와 IoT 기반의 혁신적인 가전 제품들은 어떤 것들이 있어...,"가전 넘어 모빌리티로…LG전자, AI가 바꾼 일상 그려[CES 2025]\nhttp..."


In [7]:
from langchain.docstore.document import Document

# 텍스트 분할을 위해 기본 Python 코드 사용
def split_text(text, chunk_size=1000, overlap=100):
    # 기본적으로 텍스트를 지정한 크기로 분할하는 함수
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# CSV 파일 읽기
file_path = "/content/drive/MyDrive/MINI AIFFELTHON/data/ai_articles.csv"
df = pd.read_csv(file_path)

# RAG_DB 설정: title, url, content에서 데이터 추출
RAG_DB = []

for index, row in df.iterrows():
    title = row['Title']
    url = row['URL']
    body = row['Content']  # 기사 본문

    # 텍스트를 청크로 나누기
    chunks = split_text(body)

    # 각 청크를 RAG_DB에 추가 - TAG 붙여서
    for chunk in chunks:
        RAG_DB.append({
            'title': title,
            'url': url,
            'body': chunk
        })

In [8]:
# HuggingFace 임베딩 모델
model_name = "intfloat/multilingual-e5-large-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # GPU 사용
    encode_kwargs={"normalize_embeddings": True},
)

from langchain.docstore.document import Document

# RAG_DB의 각 항목을 Document 객체로 변환
documents = [Document(page_content=article['body'], metadata={'title': article['title'], 'url': article['url']}) for article in RAG_DB]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
!pip install faiss-cpu
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [10]:
from tqdm import tqdm

# 벡터스토어 생성
documents_with_embeddings = []
for doc in tqdm(documents, desc="Processing Documents", unit="document"):
    # 각 contetn에 대해 임베딩 생성
    embedding = hf_embeddings.embed_documents([doc.page_content])[0]
    documents_with_embeddings.append((doc, embedding))

# FAISS vertor store 생성
vectorstore = FAISS.from_documents(
    documents=[doc for doc, _ in documents_with_embeddings],
    embedding=hf_embeddings
)

Processing Documents: 100%|██████████| 2613/2613 [04:04<00:00, 10.68document/s]


### 벡터 스토어 저장

In [11]:
# import faiss

# # 벡터스토어 저장 (디스크에 저장)
# save_path = '/content/drive/MyDrive/data/faiss_vectorstore'
# if not os.path.exists(save_path):
#     os.makedirs(save_path)

# # FAISS 벡터스토어를 디스크에 저장
# faiss.write_index(vectorstore.index, os.path.join(save_path, 'index.faiss'))

# # 추가적으로 메타데이터 저장 (선택 사항)
# import pickle
# with open(os.path.join(save_path, 'metadata.pkl'), 'wb') as f:
#     pickle.dump(documents, f)

### 벡터 스토어 불러오기

In [12]:
# import faiss
# import os
# import pickle

# # 저장된 벡터스토어 및 메타데이터 불러오기
# load_path = '/content/drive/MyDrive/data/faiss_vectorstore'

# # FAISS 벡터스토어 로드
# index = faiss.read_index(os.path.join(load_path, 'index.faiss'))

# # 메타데이터 로드 (선택 사항)
# with open(os.path.join(load_path, 'metadata.pkl'), 'rb') as f:
#     documents = pickle.load(f)

# # 불러온 데이터를 기반으로 FAISS 벡터스토어 객체 재생성
# vectorstore = FAISS(index=index, embedding=hf_embeddings)

# # 이제 `vectorstore`를 사용하여 검색 등의 작업을 할 수 있습니다.

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import time

# Google API Key 설정 (Gemini 모델을 사용할 때 Google API Key 필요)
YOUR_API_KEY = ''
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

# Gemini-1.5-flash 모델 설정
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# RetrievalQA 체인 생성
dense_retriever = vectorstore.as_retriever()  # FAISS 벡터스토어에서 검색 기능을 제공하는 retriever 생성

# RetrievalQA 체인 (DPR 기반 RAG)
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_gemini,
    retriever=dense_retriever,
    return_source_documents=True  # 검색된 문서도 함께 반환
)

In [14]:
# import os
# os.environ["OPENAI_API_KEY"] = "" 

In [15]:
from ragas import evaluate
from ragas.metrics import faithfulness, context_precision
from datasets import Dataset
from tqdm import tqdm
import time

# 결과를 저장할 리스트
generated_answers = []
contexts = []

# test 셋의 각 쿼리에 대해 답변 생성
for query in tqdm(test['query'], desc="Generating Answers"):
    result = dpr_qa_chain(query)
    generated_answers.append(result['result'])
    contexts.append([doc.page_content for doc in result['source_documents']])

    # 5초 대기
    time.sleep(5)

Generating Answers:   0%|          | 0/51 [00:00<?, ?it/s]<ipython-input-15-193570a3327d>:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = dpr_qa_chain(query)
Generating Answers: 100%|██████████| 51/51 [06:08<00:00,  7.23s/it]


In [18]:
!pip install --upgrade ragas

In [22]:
from ragas import evaluate
from ragas.metrics import ContextPrecision, ContextRecall, Faithfulness, AnswerRelevancy
from datasets import Dataset
import numpy as np

# RAGAS 평가를 위한 데이터셋 생성
ragas_dataset = Dataset.from_dict({
    "question": test['query'].tolist(),
    "answer": generated_answers,
    "contexts": contexts,
    "ground_truth": test['answer'].tolist()
})

# RAGAS 평가 실행
result = evaluate(
    dataset=ragas_dataset,
    metrics=[
        ContextPrecision(),
        ContextRecall(),
        Faithfulness(),
        AnswerRelevancy()
    ]
)

# 결과 출력
print(result)

# 결과를 DataFrame으로 변환
df_result = result.to_pandas()
print(df_result)

# 평균 점수 계산
numeric_columns = df_result.select_dtypes(include=[np.number]).columns
average_scores = df_result[numeric_columns].mean()

print("\n평균 점수:")
print(average_scores)

# test 데이터프레임에 생성된 답변과 평가 점수 추가
test['generated_answer'] = generated_answers
for col in numeric_columns:
    test[col] = df_result[col]

# 결과 샘플 출력 (처음 5개)
print("\n결과 샘플 (처음 5개):")
print(test[['query', 'answer', 'generated_answer'] + list(numeric_columns)].head())

Evaluating:   0%|          | 0/204 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[88]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-fd10CCJFL9AEEIplbVd2Oi6J on tokens per min (TPM): Limit 200000, Used 198611, Requested 4033. Please try again in 793ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[104]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-fd10CCJFL9AEEIplbVd2Oi6J on tokens per min (TPM): Limit 200000, Used 199293, Requested 3123. Please try again in 724ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})


{'context_precision': 1.0000, 'context_recall': 0.8947, 'faithfulness': 0.9141, 'answer_relevancy': 0.4178}
                                           user_input  \
0   LG 전자의 AI 기반 스마트 홈 기술이 CES 2025에서 어떻게 소개되었는지 알...   
1   우리나라 국방부나 군사 강국에서 AI와 IoT를 적용한 방산기술이나 트렌드에 관한 ...   
2   우리나라 기업들이 AI 기술을 활용해서 일상의 큰 변화를 일으킨 것에 대한 기사가 ...   
3   페르소나 AI는 뭐야? 또, 온디바이스 AI 기술은 뭐야? IoT는 네트워크랑 연결...   
4   LG 전자에서 발표한 AI와 IoT 기반의 혁신적인 가전 제품들은 어떤 것들이 있어...   
5    AI와 IoT를 활용한 스마트 아파트 기술이 거주자들에게 어떤 편리함을 제공하고 있어?   
6    IoT 보안 취약점과 이를 해결하기 위한 AI, 머신러닝 기술에 관한 기사를 찾아줄래?   
7            세계적으로 IoT에 AI를 접목시키는 기술 발전과 경쟁에 대해 알려줄래?   
8    AI와 IoT 를 접목시킨 신생 기업에 대해 알려줄래? 그리고 그 기업의 전망은 어때?   
9   "AI 교과서 관련 교육법 개정안에 대한 국회 논의 및 재의결 과정에 대한 기사가 있어?   
10  AI와 머신러닝을 통해 기업 비즈니스, 금융 및 제조업에서 실질적인 가치를 제공하는...   
11              교육용 디지털 기술의 발전에 AI가 활용된 사례에 대해서 알려줄래?   
12             고등학교의 AI 전문인력 양성 프로그램에 대해 다룬 기사가 있나?\n   
13  네이버가 AI를 활용해서 콘텐츠 창작자를 위한 창작 지원 프로그램을 지원한다는데 해...   
14        AI를 활용한 자동 더빙 서비스와 AI 휴먼 영상

In [24]:
test

,query,answer,generated_answer,context_precision,context_recall,faithfulness,answer_relevancy
0,LG 전자의 AI 기반 스마트 홈 기술이 CES 2025에서 어떻게 소개되었는지 알...,[CES 2025] 노트북 켰더니 AI가 알아서 '척척'… LG 제안 '공감지능'의...,LG전자는 CES 2025에서 '공감지능과 함께하는 일상의 라이프스 굿'이라는 메...,1.0,0.590909,1.000000,0.902110
1,우리나라 국방부나 군사 강국에서 AI와 IoT를 적용한 방산기술이나 트렌드에 관한 ...,"""중요한 건 AI야""... 미군과 글로벌 방산업계가 꼽았다\nhttps://n.ne...","네, 있습니다. 제공된 텍스트에는 한국과 미국을 포함한 여러 국가의 국방부 및 군사...",1.0,1.000000,1.000000,0.890551
2,우리나라 기업들이 AI 기술을 활용해서 일상의 큰 변화를 일으킨 것에 대한 기사가 ...,어쩌다 만나던 AI가 이젠 한집안 식구가 돼 가는 현실 [CES 2025]\nhtt...,제공된 텍스트에는 한국 기업들이 AI 기술을 활용하여 일상생활에 큰 변화를 일으킨 ...,1.0,1.000000,1.000000,0.000000
3,"페르소나 AI는 뭐야? 또, 온디바이스 AI 기술은 뭐야? IoT는 네트워크랑 연결...","페르소나AI, 인터넷·GPU 필요없는 ‘온디바이스 AI’로 CES 2025 혁신상 ...",제공된 텍스트에는 페르소나 AI에 대한 정보가 없습니다. 따라서 페르소나 AI가 ...,1.0,1.000000,0.750000,0.000000
4,LG 전자에서 발표한 AI와 IoT 기반의 혁신적인 가전 제품들은 어떤 것들이 있어...,"가전 넘어 모빌리티로…LG전자, AI가 바꾼 일상 그려[CES 2025]\nhttp...",LG전자가 CES 2025에서 발표한 AI와 IoT 기반의 혁신적인 가전 제품으로는...,1.0,1.000000,1.000000,0.862752
5,AI와 IoT를 활용한 스마트 아파트 기술이 거주자들에게 어떤 편리함을 제공하고 있어?,AI·IoT·로봇배송으로 진화하는 '5세대 아파트'[주간 부동산 키워드]\nhttp...,AI와 IoT를 활용한 스마트 아파트 기술은 다음과 같은 편리함을 제공합니다.\n\...,1.0,1.000000,1.000000,0.907980
6,"IoT 보안 취약점과 이를 해결하기 위한 AI, 머신러닝 기술에 관한 기사를 찾아줄래?",[전문가기고]IoT 보안 문제에 접근하는 다양한 프레임\nhttps://n.news...,제공된 텍스트에는 IoT 보안 취약점과 이를 해결하기 위한 AI 및 머신러닝 기술에...,1.0,0.800000,0.400000,0.000000
7,세계적으로 IoT에 AI를 접목시키는 기술 발전과 경쟁에 대해 알려줄래?,中 AI발전 어느 정도기에 美가 견제?…'반신반의' 넘어 '놀라움'\nhttps:/...,제공된 텍스트에는 IoT에 AI를 접목하는 기술 발전이나 경쟁에 대한 정보가 없습니...,1.0,1.000000,1.000000,0.000000
8,AI와 IoT 를 접목시킨 신생 기업에 대해 알려줄래? 그리고 그 기업의 전망은 어때?,"""AI·IoT로 데이터 혁신""…심플랫폼, 코스닥 상장 돌입\nhttps://n.ne...",제공된 텍스트에는 AI와 IoT를 접목시킨 특정 신생기업에 대한 정보가 없습니다. ...,1.0,0.833333,1.000000,0.000000
9,"""AI 교과서 관련 교육법 개정안에 대한 국회 논의 및 재의결 과정에 대한 기사가 있어?","AI 교과서 '교육자료 격하법'에 거부권…""교육격차 심화""\nhttps://n.ne...","네, 있습니다. 기사에 따르면 야당 주도로 AI 교과서의 법적 지위를 '교과서'가...",1.0,0.812500,1.000000,0.000000


In [ ]:
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.metrics import precision_score, recall_score

# # n-gram Precision 계산 함수
# def calculate_ngram_precision(actual, generated, n=1):
#     actual_ngrams = set(zip(*[actual.split()[i:] for i in range(n)]))
#     generated_ngrams = set(zip(*[generated.split()[i:] for i in range(n)]))

#     # 교집합을 구하여 precision 계산
#     intersection = len(actual_ngrams.intersection(generated_ngrams))
#     precision = intersection / len(generated_ngrams) if len(generated_ngrams) > 0 else 0
#     return precision

# # n-gram Recall 계산 함수
# def calculate_ngram_recall(actual, generated, n=1):
#     actual_ngrams = set(zip(*[actual.split()[i:] for i in range(n)]))
#     generated_ngrams = set(zip(*[generated.split()[i:] for i in range(n)]))

#     # 교집합을 구하여 recall 계산
#     intersection = len(actual_ngrams.intersection(generated_ngrams))
#     recall = intersection / len(actual_ngrams) if len(actual_ngrams) > 0 else 0
#     return recall

# # Cosine Similarity 계산 함수 (faithfulness 및 answer_relevancy에 사용)
# def calculate_cosine_similarity(text1, text2):
#     vectorizer = CountVectorizer().fit_transform([text1, text2])
#     vectors = vectorizer.toarray()
#     return cosine_similarity(vectors)[0][1]

# # RAGas 평가 지표 계산 함수
# def calculate_ragas_metrics(answer, generated_answer, query, n=1):
#     # Cosine Similarity로 'faithfulness'와 'answer_relevancy' 계산
#     faithfulness = calculate_cosine_similarity(answer, generated_answer)
#     answer_relevancy = calculate_cosine_similarity(query, generated_answer)

#     # n-gram precision과 recall 계산
#     context_precision = calculate_ngram_precision(answer, generated_answer, n)
#     context_recall = calculate_ngram_recall(answer, generated_answer, n)

#     return context_precision, faithfulness, context_recall, answer_relevancy

# # 결과 계산
# results = []
# for index, row in tqdm(test.iterrows(), desc="Processing Test Set", total=len(test)):
#     answer = row['answer']
#     generated_answer = row['generated_answer']
#     query = row['query']

#     context_precision, faithfulness, context_recall, answer_relevancy = calculate_ragas_metrics(answer, generated_answer, query)

#     results.append({
#         'query': query,
#         'answer': answer,
#         'generated_answer': generated_answer,
#         'context_precision': context_precision,
#         'faithfulness': faithfulness,
#         'context_recall': context_recall,
#         'answer_relevancy': answer_relevancy
#     })

# # 결과를 DataFrame으로 저장
# results_df = pd.DataFrame(results)

# # 결과 출력
# print(results_df)

Processing Test Set: 100%|██████████| 51/51 [00:00<00:00, 131.82it/s]

                                                query  \
0   LG 전자의 AI 기반 스마트 홈 기술이 CES 2025에서 어떻게 소개되었는지 알...   
1   우리나라 국방부나 군사 강국에서 AI와 IoT를 적용한 방산기술이나 트렌드에 관한 ...   
2   우리나라 기업들이 AI 기술을 활용해서 일상의 큰 변화를 일으킨 것에 대한 기사가 ...   
3   페르소나 AI는 뭐야? 또, 온디바이스 AI 기술은 뭐야? IoT는 네트워크랑 연결...   
4   LG 전자에서 발표한 AI와 IoT 기반의 혁신적인 가전 제품들은 어떤 것들이 있어...   
5    AI와 IoT를 활용한 스마트 아파트 기술이 거주자들에게 어떤 편리함을 제공하고 있어?   
6    IoT 보안 취약점과 이를 해결하기 위한 AI, 머신러닝 기술에 관한 기사를 찾아줄래?   
7            세계적으로 IoT에 AI를 접목시키는 기술 발전과 경쟁에 대해 알려줄래?   
8    AI와 IoT 를 접목시킨 신생 기업에 대해 알려줄래? 그리고 그 기업의 전망은 어때?   
9   "AI 교과서 관련 교육법 개정안에 대한 국회 논의 및 재의결 과정에 대한 기사가 있어?   
10  AI와 머신러닝을 통해 기업 비즈니스, 금융 및 제조업에서 실질적인 가치를 제공하는...   
11              교육용 디지털 기술의 발전에 AI가 활용된 사례에 대해서 알려줄래?   
12             고등학교의 AI 전문인력 양성 프로그램에 대해 다룬 기사가 있나?\n   
13  네이버가 AI를 활용해서 콘텐츠 창작자를 위한 창작 지원 프로그램을 지원한다는데 해...   
14        AI를 활용한 자동 더빙 서비스와 AI 휴먼 영상 제작에 관한 기사가 있을까?   
15  AI 기술을 활용한 의료, 교육, 금융 등 분야의 기업들이 포용성 증대를 활용한 사...   
16    우리나라 정부에서 AI를 활용한 정보화 교육 

In [ ]:
# results_df

,query,answer,generated_answer,context_precision,faithfulness,context_recall,answer_relevancy
0,LG 전자의 AI 기반 스마트 홈 기술이 CES 2025에서 어떻게 소개되었는지 알...,[CES 2025] 노트북 켰더니 AI가 알아서 '척척'… LG 제안 '공감지능'의...,LG전자는 CES 2025에서 '공감지능과 함께하는 일상의 라이프스 굿'이라는 메...,0.444444,0.590690,0.142132,0.370630
1,우리나라 국방부나 군사 강국에서 AI와 IoT를 적용한 방산기술이나 트렌드에 관한 ...,"""중요한 건 AI야""... 미군과 글로벌 방산업계가 꼽았다\nhttps://n.ne...","네, 제공된 텍스트에는 한국 국방부와 군사 강국에서 AI와 IoT를 적용한 방산 기...",0.326923,0.509334,0.129442,0.098601
2,우리나라 기업들이 AI 기술을 활용해서 일상의 큰 변화를 일으킨 것에 대한 기사가 ...,어쩌다 만나던 AI가 이젠 한집안 식구가 돼 가는 현실 [CES 2025]\nhtt...,제공된 텍스트에는 한국 기업들이 AI 기술을 활용하여 일상생활에 큰 변화를 일으킨 ...,0.125000,0.165748,0.009231,0.377964
3,"페르소나 AI는 뭐야? 또, 온디바이스 AI 기술은 뭐야? IoT는 네트워크랑 연결...","페르소나AI, 인터넷·GPU 필요없는 ‘온디바이스 AI’로 CES 2025 혁신상 ...",제공된 텍스트에는 페르소나 AI에 대한 정보가 없습니다. 따라서 페르소나 AI가 ...,0.156250,0.199273,0.054945,0.187083
4,LG 전자에서 발표한 AI와 IoT 기반의 혁신적인 가전 제품들은 어떤 것들이 있어...,"가전 넘어 모빌리티로…LG전자, AI가 바꾼 일상 그려[CES 2025]\nhttp...",LG전자는 CES 2025에서 AI와 IoT 기반의 다양한 가전 제품들을 선보였습니...,0.460227,0.542485,0.134551,0.242480
5,AI와 IoT를 활용한 스마트 아파트 기술이 거주자들에게 어떤 편리함을 제공하고 있어?,AI·IoT·로봇배송으로 진화하는 '5세대 아파트'[주간 부동산 키워드]\nhttp...,AI와 IoT를 활용한 스마트 아파트 기술은 다음과 같은 편리함을 제공합니다.\n\...,0.377622,0.314361,0.215139,0.187168
6,"IoT 보안 취약점과 이를 해결하기 위한 AI, 머신러닝 기술에 관한 기사를 찾아줄래?",[전문가기고]IoT 보안 문제에 접근하는 다양한 프레임\nhttps://n.news...,제공된 텍스트에는 IoT 보안 취약점과 이를 해결하기 위한 AI 및 머신러닝 기술에...,0.193548,0.371105,0.034286,0.540738
7,세계적으로 IoT에 AI를 접목시키는 기술 발전과 경쟁에 대해 알려줄래?,中 AI발전 어느 정도기에 美가 견제?…'반신반의' 넘어 '놀라움'\nhttps:/...,제공된 텍스트에는 IoT에 AI를 접목하는 기술 발전이나 경쟁에 대한 정보가 없습니...,0.235294,0.055350,0.009524,0.290957
8,AI와 IoT 를 접목시킨 신생 기업에 대해 알려줄래? 그리고 그 기업의 전망은 어때?,"""AI·IoT로 데이터 혁신""…심플랫폼, 코스닥 상장 돌입\nhttps://n.ne...",제공된 텍스트에는 AI와 IoT를 접목시킨 특정 신생기업에 대한 정보가 없습니다. ...,0.000000,0.000000,0.000000,0.181818
9,"""AI 교과서 관련 교육법 개정안에 대한 국회 논의 및 재의결 과정에 대한 기사가 있어?","AI 교과서 '교육자료 격하법'에 거부권…""교육격차 심화""\nhttps://n.ne...","네, 있습니다. 기사에 따르면 야당 주도로 AI 교과서의 법적 지위를 '교과서'가...",0.390625,0.323151,0.094697,0.152145


In [ ]:
# results_df.iloc[:,3:7]

,context_precision,faithfulness,context_recall,answer_relevancy
0,0.444444,0.590690,0.142132,0.370630
1,0.326923,0.509334,0.129442,0.098601
2,0.125000,0.165748,0.009231,0.377964
3,0.156250,0.199273,0.054945,0.187083
4,0.460227,0.542485,0.134551,0.242480
5,0.377622,0.314361,0.215139,0.187168
6,0.193548,0.371105,0.034286,0.540738
7,0.235294,0.055350,0.009524,0.290957
8,0.000000,0.000000,0.000000,0.181818
9,0.390625,0.323151,0.094697,0.152145


In [ ]:
# context_precision    0.352605
# faithfulness         0.312957
# context_recall       0.081066
# answer_relevancy     0.265077
# dtype: float64